# Summarizer

In this example we will use Guardrails to summarize text in terms of length, quality and output read time.

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/summarization.ipynb).

In this example, we will use Guardrails in the summarization of a text document. We will check whether the summarized document has a high semantic similarity with the original document. We will ensure the ouptput meets a certain range of length and read time.

## Setup
In order to check semantic similarity we will need the `numpy` package via the command below. We will also install the validators we indtend to use.

In [1]:
%pip install numpy -q
! guardrails hub install hub://guardrails/reading_time --quiet --install-local-models
! guardrails hub install hub://guardrails/similar_to_document --quiet --install-local-models
! guardrails hub install hub://guardrails/valid_length --quiet --install-local-models


[notice] A new release of pip is available: 23.2.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Installing hub://guardrails/reading_time...
✅Successfully installed guardrails/reading_time!


Installing hub://guardrails/similar_to_document...
✅Successfully installed guardrails/similar_to_document!


Installing hub://guardrails/valid_length...
✅Successfully installed guardrails/valid_length!




## Step 1: Load data and create Pydantic Model

Load our text with the code below

In [2]:
with open("data/twain.txt", "r") as file:
    document = file.read()
    file.seek(0)
    content = "".join(line.strip() for line in file.readlines())

Next we can define our return output with a pydantic model

In [ ]:
from pydantic import BaseModel, Field

from guardrails.hub import SimilarToDocument, ValidLength, ReadingTime

prompt = """
Summarize the following text faithfully:

${document}

${gr.complete_xml_suffix}
"""

THREE_MINUTES = 180 / 60

class TextSummary(BaseModel):
    summary: str = Field(
        description="Faithful summary of the text",
        validators=[
            ReadingTime(reading_time=THREE_MINUTES, on_fail="exception"),
            ValidLength(min=100, max=1000, on_fail="exception"),
            SimilarToDocument(document=f"'{content}'", threshold=0.60, on_fail="filter")
        ],
    )

## Step 2 Create Guard from pydantic

The guard we create will:
1. Enforce reading time
2. Enforce length
3. Enforce similarity

In [4]:
import guardrails as gd
from guardrails.errors import ValidationError

guard = gd.Guard().for_pydantic(TextSummary)

## Step 3: Call LLM via `guard(`

We use the tools api to ensure our data is returned in a structured form. 


In [5]:
import os
# TODO: Replace OPENAI_API_KEY with your OpenAI API key, uncomment
# os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

response = guard(
    messages=[{"role":"user", "content": prompt}],
    prompt_params={"document": document},
    model="gpt-4o",
    tools=guard.json_function_calling_tool(),
    tool_choice="required",
)

print(f"Validated Output: {response.validated_output}")

Similarity: 0.789, Type: <class 'float'>
Validated Output: {'summary': "Mark Twain discusses the art of storytelling, focusing on the humorous story, which he considers uniquely American. He contrasts it with the comic story, which is English, and the witty story, which is French. The humorous story relies on the manner of telling, can be lengthy, and often ends without a clear point. In contrast, comic and witty stories are brief and end with a punchline. Twain emphasizes that telling a humorous story requires high artistic skill, while anyone can tell comic or witty stories. He notes that humorous stories are told gravely, with the teller pretending not to know it's funny, whereas comic stories are told with obvious delight and emphasis on the punchline. Twain mentions that some storytellers, like Artemus Ward, use subtle techniques to deliver the punchline, while comic storytellers make it very obvious. He finds the latter approach less appealing."}


We can see the step-wise history of the `Guard` object below:

In [6]:
guard.history.last.tree

Logs
└── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
    │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
    │ │ No prompt                                                                                               │ │
    │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
    │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
    │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
    │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
    │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
    │ │ │ user │                                                                                              │ │ │
    │ │ │      │ Summarize the following text faithfully:                                                     │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ The Humorous Story an American Development.— Its                                             │ │ │
    │ │ │      │ Difference from Comic and Witty Stories.                                                     │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ DO not claim that I can tell a story as it ought to                                          │ │ │
    │ │ │      │ be told. I only claim to know how a story                                                    │ │ │
    │ │ │      │ ought to be told, for I have been almost daily in the                                        │ │ │
    │ │ │      │ company of the most expert story-tellers for many                                            │ │ │
    │ │ │      │ years.                                                                                       │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ There are several kinds of stories, but only one                                             │ │ │
    │ │ │      │ difficult kind —the humorous. I will talk mainly                                             │ │ │
    │ │ │      │ about that one. The humorous story is American,                                              │ │ │
    │ │ │      │ the comic story is English, the witty story is French.                                       │ │ │
    │ │ │      │ The humorous story depends for its effect upon the                                           │ │ │
    │ │ │      │ manner of the telling; the comic story and the witty                                         │ │ │
    │ │ │      │ story upon the matter.                                                                       │ │ │
    │ │ │      │                                                                                              │ │ │
    │ │ │      │ The humorous story may be spun out to great                                                  │ │ │
    │ │ │      │ length, and may wander around as much as it                                                  │ │ │
    │ │ │      │ pleases, and arrive nowhere in particular; but the                                           │ │ │
    │ │ │      │ comic and witty stories must be brief and end with                                           │ │ │
    │ │ │      │ a point. The humorous story bubbles gently along,                                            │ │ │
    │ │ │      │ the others burst.                                                                            │ │ │
    │ │ │      │                                   

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary). We can see that the output is a dictionary with the correct schema and types.

Now lets try a model thats not as proficent at summarization and we can see the ouptput is filtered and validation has failed.

The final validated output is `None` due to the failed validation.

In [7]:
response = guard(
    messages=[{"role":"user", "content": prompt}],
    prompt_params={"document": document},
    model="babbage-002",
    max_tokens=512,
    temperature=0,
)

print(f"Validated Output: {response.validated_output}")

Validated Output: None


We can see the step wise history of the guard execution below:

In [8]:
guard.history.last.tree

Logs
├── ╭────────────────────────────────────────────────── Step 0 ───────────────────────────────────────────────────╮
│   │ ╭──────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────╮ │
│   │ │ No prompt                                                                                               │ │
│   │ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│   │ ╭──────────────────────────────────────────── Message History ────────────────────────────────────────────╮ │
│   │ │ ┏━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓ │ │
│   │ │ ┃ Role ┃ Content                                                                                      ┃ │ │
│   │ │ ┡━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩ │ │
│   │ │ │ user │                                                                                              │ │ │
│   │ │ │      │ Summarize the following text faithfully:                                                     │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ The Humorous Story an American Development.— Its                                             │ │ │
│   │ │ │      │ Difference from Comic and Witty Stories.                                                     │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ DO not claim that I can tell a story as it ought to                                          │ │ │
│   │ │ │      │ be told. I only claim to know how a story                                                    │ │ │
│   │ │ │      │ ought to be told, for I have been almost daily in the                                        │ │ │
│   │ │ │      │ company of the most expert story-tellers for many                                            │ │ │
│   │ │ │      │ years.                                                                                       │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ There are several kinds of stories, but only one                                             │ │ │
│   │ │ │      │ difficult kind —the humorous. I will talk mainly                                             │ │ │
│   │ │ │      │ about that one. The humorous story is American,                                              │ │ │
│   │ │ │      │ the comic story is English, the witty story is French.                                       │ │ │
│   │ │ │      │ The humorous story depends for its effect upon the                                           │ │ │
│   │ │ │      │ manner of the telling; the comic story and the witty                                         │ │ │
│   │ │ │      │ story upon the matter.                                                                       │ │ │
│   │ │ │      │                                                                                              │ │ │
│   │ │ │      │ The humorous story may be spun out to great                                                  │ │ │
│   │ │ │      │ length, and may wander around as much as it                                                  │ │ │
│   │ │ │      │ pleases, and arrive nowhere in particular; but the                                           │ │ │
│   │ │ │      │ comic and witty stories must be brief and end with                                           │ │ │
│   │ │ │      │ a point. The humorous story bubbles gently along,                                            │ │ │
│   │ │ │      │ the others burst.                                                                            │ │ │
│   │ │ │      │                                   